In [1]:
using Pkg
Pkg.activate(@__DIR__);
Pkg.instantiate()

using StatsPlots
using CSV, DataFrames

import Dates: DateTime
import Base: tryparse

  Activating project at `c:\Users\ayats\Documents\ici-parsing`


In [6]:
@enum VarKind begin
    Hum
    Pres
    Temp
end


function tryparse(::Type{VarKind}, s::String)
    if s == "6"
        Hum::VarKind
    elseif s == "7"
        Pres::VarKind
    elseif s == "11"
        Temp::VarKind
    else
        nothing
    end
end


data = CSV.File(
    open("data/Montecoche_Marzo2023.csv");
    normalizenames=true,
    select=[:ts, :value, :var_id],
    types=Dict(:ts => DateTime, :var_id => VarKind),
    dateformat="yyyy-mm-dd HH:MM:SS.s"
) |> DataFrame

Row,var_id,value,ts
,VarKind,Float64,DateTime
1,Hum,76.6,2023-03-01T00:05:01
2,Hum,84.3,2023-03-01T00:24:56
3,Hum,87.3,2023-03-01T00:35:20
4,Hum,100.0,2023-03-01T01:24:33
5,Hum,100.0,2023-03-01T02:55:42
6,Hum,100.0,2023-03-01T03:15:01
7,Hum,100.0,2023-03-01T03:24:33
8,Hum,100.0,2023-03-01T03:34:51
9,Hum,95.7,2023-03-01T03:46:02


In [34]:
using Query

data |> 
    @filter(_.var_id == Hum::VarKind) |> 
    @df plot(:ts, :value)

In [33]:
data |> 
    @filter(_.var_id == Hum::VarKind) |> 
    @filter(_.value <= 100.0) |>
    @filter(_.value >= 0.0) |>
    @df plot(:ts, :value, marker=:circle, markersize=2)

ylims!((-10, 110))